# Prediction of Solar power

Data provided open source at:
https://www.kaggle.com/datasets/dilipkola/shell-ai-solar-irradiance-prediction-hackathon?group=bookmarked

##### Goals of analysis:
* Predict Global Irradiance for next 2 hours at 10 minute intervals given at least 2 hours of weather data

##### Data:
* Data is given every minute meaning at least 120 observations (2hours) must be used to output 12 data points (10 minute increments)

### Housekeeping

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import datetime as dt
import os
import sys
from Settings import columns, experiments
import Pipeline
# from sklearn.model_selection import train_test_split
import random
import tensorflow as tf

In [2]:
os.getcwd() in sys.path


True

In [3]:
train = pd.read_csv('./archive/train.csv')
test = pd.read_csv('./archive/test.csv')

### Data Exploration
##### Thoughts while data cleaning
* Wet bulb is measure of heat stress in direct sunlight -- mixture of temp, humidity, wind speed, sun angle, and cloud cover
    * (Would be interesting to look at the change in dWB/dt)
* Would be interesting to take out yearly cycle, not sure if I have enough data. 

In [4]:
pipeline = Pipeline.DataClean(columns, experiments['all'])
pipeline.clean_data()
pipeline.norm()
pipeline.split_label()
# pipeline.train_val()

Right now I am splitting my train and validation dataset through a random sampler. I will just grab data in 120 chunks and move on to try to get the model working.

In [21]:
pipeline.label.sample(1000, replace=False, random_state=26)

25943       0.000000
88496     734.708000
220023     12.720200
430742      0.000000
23159       0.000000
             ...    
88903      14.476600
256040      0.582808
203039      0.000000
109626      0.000000
436436      0.000000
Name: label, Length: 1000, dtype: float64

In [57]:
def itterlabel():
    for i in range(2):
        yield pipeline.label.sample(120, replace=False)
        
def itterdata():
    for i in range(2):
        yield pipeline.data.sample(120, replace=False)


In [73]:
pd.DataFrame(list(itterdata())[0])

,Global CMP22 (vent/cor) [W/m^2],Tower Wet Bulb Temp [deg C],Direct sNIP [W/m^2],Azimuth Angle [degrees],Tower Dew Point Temp [deg C],Tower RH [%],Total Cloud Cover [%],Peak Wind Speed @ 6ft [m/s],Avg Wind Direction @ 6ft [deg from N],Station Pressure [mBar],Precipitation (Accumulated) [mm],Snow Depth [cm],Moisture,Albedo (CMP11)
393890,0.343496,0.511434,0.714110,0.652446,0.000000,0.190114,0.22,0.039256,0.049639,0.541050,0.0,0.004467,0.0,0.11515
378203,0.021518,0.572054,0.001713,0.738253,0.000000,0.124831,0.52,0.214876,0.697778,0.388425,0.0,0.000000,0.0,0.08210
433150,0.000000,0.000000,0.000079,0.825082,0.000000,0.537804,0.00,0.152893,0.812222,0.422622,0.0,0.181850,0.0,0.00000
271076,0.400460,0.830501,0.824777,0.245979,0.500919,0.251017,0.13,0.173554,0.247222,0.403620,0.0,0.029581,0.0,0.10135
220817,0.601780,0.696333,0.278790,0.337063,0.150419,0.156429,0.99,0.173554,0.224444,0.486153,0.0,0.038305,0.0,0.08485
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
491744,0.288406,0.173362,0.921714,0.577197,0.000000,0.056315,0.23,0.080579,0.193972,0.547724,0.0,0.012452,0.0,0.12600
416752,0.384118,0.289761,0.485777,0.505360,0.000000,0.164146,0.54,0.111570,0.416389,0.421416,0.0,0.014562,0.0,0.10855
161858,0.188647,0.346158,0.009325,0.468991,0.000000,0.253103,0.95,0.173554,0.084472,0.378075,0.0,0.042737,0.0,0.08960
451346,0.031203,0.000000,0.000739,0.532564,0.000000,0.980186,0.97,0.152893,0.111417,0.275879,0.0,0.000000,1.0,0.11520


In [15]:
tf.data.Dataset.from_tensors('./archive/train.csv')

tensorflow.python.data.ops.dataset_ops.TensorDataset

In [85]:
label = tf.data.Dataset.from_generator(itterlabel, output_types=tf.float32, output_shapes=(120,),)
data = tf.data.Dataset.from_generator(itterdata, output_types=tf.float32, output_shapes=(120,14,),)



tensorflow.python.data.ops.dataset_ops.FlatMapDataset

In [107]:


model = tf.keras.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10)
])

model.compile(optimizer='adam',
             loss=tf.keras.losses.MeanSquaredError(),
             metrics=['accuracy'])

together = tf.data.Dataset.from_tensor_slices((data,label))
together = dataset.batch(10).repeat()
inputs = Input(shape=(14,))
type(together)

# for count_batch in together.repeat().batch(1).take(1):
#   print(count_batch)


model.fit(together.mae_one_shot_iterator(), epochs=2)

ValueError: Slicing dataset elements is not supported for rank 0.

#### For tomorrow:
* try yeild function in notebook to determine how long it runs to see if it is more efficient

In [5]:
pipeline.data.sample(n=120, replace=False, random_state=26)

,Global CMP22 (vent/cor) [W/m^2],Tower Wet Bulb Temp [deg C],Direct sNIP [W/m^2],Azimuth Angle [degrees],Tower Dew Point Temp [deg C],Tower RH [%],Total Cloud Cover [%],Peak Wind Speed @ 6ft [m/s],Avg Wind Direction @ 6ft [deg from N],Station Pressure [mBar],Precipitation (Accumulated) [mm],Snow Depth [cm],Moisture,Albedo (CMP11)
25943,0.000000,0.000000,0.000375,0.181309,0.000000,0.419648,0.00,0.080579,0.800833,0.554190,0.000000,0.006542,0.0,0.00000
88496,0.546643,0.068140,0.947409,0.540655,0.000000,0.286370,0.13,0.049587,0.371944,0.204487,0.108123,0.032149,0.0,0.09220
220023,0.000000,0.685414,0.000000,0.883766,0.373636,0.303264,0.00,0.121901,0.790278,0.446640,0.000000,0.033943,0.0,0.00000
430742,0.000000,0.000000,0.000000,0.258891,0.000000,0.857128,0.00,0.090909,0.052111,0.475766,0.000000,0.414351,0.0,0.00000
23159,0.000000,0.000000,0.000000,0.239548,0.000000,0.489728,0.00,0.059917,0.040389,0.255123,0.000000,0.003869,0.0,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205513,0.000793,0.396477,0.000000,0.826922,0.000000,0.088539,0.00,0.000000,0.000000,0.251089,0.000000,0.024974,0.0,0.00000
380786,0.636056,0.677740,0.755484,0.535183,0.078023,0.137971,0.67,0.163223,0.270833,0.482816,0.000000,0.014351,0.0,0.09675
499577,0.000000,0.000000,0.000203,0.048154,0.000000,0.935343,0.00,0.163223,0.085278,0.429541,0.000000,0.127365,1.0,0.00000
135534,0.000000,0.000000,0.000139,0.205867,0.000000,0.690583,0.00,0.059917,0.836944,0.339316,0.000000,0.031551,0.0,0.00000


In [6]:
# sns.heatmap(train.corr())

In [7]:
pipeline.data.columns.shape

(14,)

In [8]:
pipeline.data

,Global CMP22 (vent/cor) [W/m^2],Tower Wet Bulb Temp [deg C],Direct sNIP [W/m^2],Azimuth Angle [degrees],Tower Dew Point Temp [deg C],Tower RH [%],Total Cloud Cover [%],Peak Wind Speed @ 6ft [m/s],Avg Wind Direction @ 6ft [deg from N],Station Pressure [mBar],Precipitation (Accumulated) [mm],Snow Depth [cm],Moisture,Albedo (CMP11)
0,0.0,0.044602,0.000000,0.178902,0.0,0.281885,0.0,0.266529,0.729444,0.201621,0.0,0.009075,0.0,0.0
1,0.0,0.047899,0.000000,0.179747,0.0,0.280843,0.0,0.204545,0.723611,0.201602,0.0,0.005522,0.0,0.0
2,0.0,0.050062,0.000005,0.180584,0.0,0.281781,0.0,0.173554,0.747500,0.202394,0.0,0.004713,0.0,0.0
3,0.0,0.045581,0.000000,0.181415,0.0,0.282094,0.0,0.204545,0.755556,0.201527,0.0,0.004115,0.0,0.0
4,0.0,0.046714,0.000000,0.182239,0.0,0.287308,0.0,0.256198,0.766667,0.201565,0.0,0.007598,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
526915,0.0,0.000000,0.000000,0.979529,0.0,0.410887,0.0,0.000000,0.000000,0.400943,0.0,0.101970,0.0,0.0
526916,0.0,0.000000,0.000000,0.981720,0.0,0.412556,0.0,0.000000,0.000000,0.400924,0.0,0.100809,0.0,0.0
526917,0.0,0.000000,0.000000,0.983916,0.0,0.412139,0.0,0.000000,0.000000,0.401169,0.0,0.101372,0.0,0.0
526918,0.0,0.000000,0.000000,0.986117,0.0,0.416728,0.0,0.000000,0.000000,0.401093,0.0,0.098663,0.0,0.0


In [9]:
np.shape(train_test_split(pipeline.data, test_size=0.3, train_size=0.7))

NameError: name 'train_test_split' is not defined

In [ ]:
train_test_split([1,2,3,3,4,5,6,7,5,6,4], test_size=0.3, train_size=0.7)

I need to split train/val by grabbing chunks of 120 samples in a row

In [ ]:
len(pipeline.data)

In [ ]:
startidx = []
samples = []
numsamples = 120
for i in range(60):
    startidx.append(random.choice(pipeline.data.index[:-numsamples]))
    samples.append(pipeline.data[startidx[i]:startidx[i]+numsamples])

In [ ]:
np.asarray(samples).shape

In [ ]:
60*0.7

In [ ]:
np.shape(samples[:42])

In [ ]:
np.shape(samples[42:])

In [ ]:
42+18